# Problem need to be addressed 

In this research, we proposal a new fault tolerance analysis technique - fault tolerance boundary is designed specific to the data variables.

why the huristic work, or why it does not work.


# Can you think a neural network method to understand program's property?
# Understand the propagation value of the nearest neighbor.
# Sampling method



1. What's the fault tolerance boundary.
2. It's a technique can be equivlent to the fault injection campaign.
3. We propose a method to approximate the boundary.
    1. heuristic method to measure the boundary.
    2. correlation analysis between variables to understand the relationship between the fault tolerance across variables.
4. label propagation and active learning.
5. The why masked prediction fail also can we use the SDC run to predict the program's outcome

In [5]:
import numpy as np
import seaborn as sns
import pandas as pd
import os.path
import matplotlib.pyplot as plt
import random
import math
from IPython.core.debugger import set_trace
from datetime import datetime
import sys
random.seed(datetime.now())

In [53]:
DATASET = "lu" #current available datasets fft, lu, cg, other potential dataset.,

In [54]:
# Threshold that used to decide whether the final outcome is SDC or Masked. For the loading data
# set, there are two different datasets. One is the exhaust fault injection campaign which contains
# all the fault injection information. The other is the test data set which is the sample experiment 
# contain the detail propagation information used for the downstream propagation analysis.
PROPAGATION_DATA_PATH = ""
CAMPAIGN_DATA_PATH = ""
PROPAGATION_INJECTION_DATA_PATH = ""
THRESHOLD = 0

if DATASET == "fft":
    PROPAGATION_DATA_PATH = "../static/data/fft/fft_complete"
    CAMPAIGN_DATA_PATH = "../static/data/fft/fft_injectlog.log"
    THRESHOLD = 0.001
elif DATASET == "cg":
    PROPAGATION_DATA_PATH = "../static/data/cg/cg_in8"
    CAMPAIGN_DATA_PATH = "../static/data/cg/cg_in8/injectlog.log"
    THRESHOLD = 0.06
elif DATASET == "lu":
    PROPAGATION_DATA_PATH = "../static/data/lu/lu20000"
    CAMPAIGN_DATA_PATH = "../static/data/lu/injectlog.log"
    THRESHOLD = 0.0001
elif DATASET == "bs":
    PROPAGATION_DATA_PATH = "../static/data/bs/bs"
    CAMPAIGN_DATA_PATH = "../static/data/bs/injectlog_complete.log"
    THRESHOLD = 0.0001
     
# The fault injection campaign experiment
FAULT_INJECTION_CAMPAIGN = pd.read_csv(CAMPAIGN_DATA_PATH,  sep=" ", names=['fileindex', 'file', 'linenum', 'variable','byte_num', 'corrupted','init_value','to', 'corrupt_value', 'mask', 'byte','expo', 'ss', 'op', 'diffnorm', 'empty'])

# The size of the fault injection experiment
SIZE = len(FAULT_INJECTION_CAMPAIGN)

# The percentage of the test experiment over exhaust fault injection campaign.
TEST_EXPERIMENT_NUMBER = int(SIZE * 0.01)

# Golden Run
GOLDEN_RUN = pd.read_csv(PROPAGATION_DATA_PATH+"/golden.log",  sep=" ", names=['file', 'linenum', 'variable', 'value'])
TEST_EXPERIMENTS = pd.read_csv(PROPAGATION_DATA_PATH+"/injectlog.log",  sep=" ", names=['fileindex', 'file', 'linenum', 'variable','byte_num', 'corrupted','init_value','to', 'corrupt_value', 'mask', 'byte','expo', 'ss', 'op', 'diffnorm', 'empty'])

if len(TEST_EXPERIMENTS) > 1000:
    TEST_EXPERIMENTS = TEST_EXPERIMENTS.sample(n=1000, replace=True, random_state=1)

# SDC ratio of test experiments
SDC_COUNT = 0
for index, row in FAULT_INJECTION_CAMPAIGN.iterrows():
    if float(row['diffnorm']) > THRESHOLD:
        SDC_COUNT += 1

print("The SDC ratio in this experiment is",SDC_COUNT/len(FAULT_INJECTION_CAMPAIGN))
print("The size of the experiment is",  len(TEST_EXPERIMENTS))
print("The size of the exhaust fault injection experiments", len(FAULT_INJECTION_CAMPAIGN))

C:\Users\zhimin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The SDC ratio in this experiment is 0.31746922896182767
The size of the experiment is 1000
The size of the exhaust fault injection experiments 885248


# Golden Fault Injection Boundary
Golden fault injection boundary use the exhaust fault inject campaign to build the boundary in practical sense. If the error value is above a specific threshold, then the location will not tolerance the error.

In [55]:
# In fault injection campaign analysis, the prediction outcome is positive and negative.
# during the prediction process, we ignore the crash case.
# For the prediction, we need to think about four different cases
# True negative, False positive
# False negative, True positive
def predict(boundary):
    positive_prediction = 0
    crash = 0
    negative_prediction = 0
    result = {}
    bits = []
    nan = float('nan')
    inf = float('inf')
    
    mitigation_set = {}
    count = 0

    for index, row in FAULT_INJECTION_CAMPAIGN.iterrows():
        instruction_index = int(row["byte_num"].split("#")[1])
        inject_error = 0
        corrupt_value = float(row['corrupt_value'])
        init_value = float(row['init_value'])
        
        if init_value != 0 and corrupt_value != nan and corrupt_value != inf:
            #inject_error = abs((corrupt_value - init_value)/init_value)
            inject_error = abs(corrupt_value - init_value)
        
        diffnorm = float(row['diffnorm'])
        
        #if boundary[instruction_index] > 0 and inject_error >
            
        
        if inject_error < boundary[instruction_index]:
            #bits.append(row["bit"])
            if diffnorm < THRESHOLD:
                positive_prediction += 1
            elif math.isnan(diffnorm) or math.isinf(diffnorm):
                crash += 1
                #print(row)
                #print(row['linenum'],row['variable'],row['corrupt_value'], row['init_value'])
            else:
                negative_prediction += 1 
                #print(row)
         
        if inject_error < boundary[instruction_index] and diffnorm > 10:
            key = str(row['linenum']) + row['variable']
            print(row['fileindex'])
            #if key in mitigation_set:
            #    mitigation_set[key] += 1
            #else:
            #    mitigation_set[key] = 1          
    
  
    result["positive"] = positive_prediction
    result["negative"] = negative_prediction
    result["crash"] = crash
    result["accuracy"] = positive_prediction/(positive_prediction + negative_prediction)
    
    return (result, bits)

def getGoldenFaultToleranceBoundary():
    boundary = {}
    nan = float('nan')
    inf = float('inf')
    for index, row in FAULT_INJECTION_CAMPAIGN.iterrows():
        corrupt_value = float(row['corrupt_value'])
        init_value = float(row['init_value'])
        inject_error = 0
        norm = float(row["diffnorm"])
        
        try:
            instruction_index = int(row["byte_num"].split("#")[1])
        except:
            print(row)
        
        if init_value != 0 and corrupt_value != nan and corrupt_value != inf:
            inject_error = abs((corrupt_value - init_value)/init_value)
  
        if instruction_index not in boundary:
            boundary[instruction_index] = float("+inf")
        
        if norm > THRESHOLD:
            if boundary[instruction_index] > inject_error:
                boundary[instruction_index] = inject_error   
        
    return boundary

#boundary = getGoldenFaultToleranceBoundary()
#print(predict(boundary)[0])

# Inference Evaluation
1. We uniformly random sample 1000 experiments and use the masked cases to predict the first 5, 10, 20, 50, 100, 200, 300 corrupted instructions' outcome.

In [56]:
experiment_set=[5, 10, 20, 50, 100, 200, 300]
l = len(GOLDEN_RUN)

for knn in experiment_set:
    
    prediction_accuracy = []
    prediction_amount = []
    for index, row in TEST_EXPERIMENTS.iterrows():


        #Ignore the SDC cases
        if float(row["diffnorm"]) > THRESHOLD:
            continue

        #
        fault_inject_run = pd.read_csv(PROPAGATION_DATA_PATH+"/appstate_"+str(index)+".log",  sep=' ', names=['file', 'linenum', 'variable', 'value'])
        if len(fault_inject_run) < l:
            print("bad case!")
            continue
        else:
            fault_inject_run_value = np.array(fault_inject_run.value, dtype='float')[0:l] - np.array(GOLDEN_RUN.value, dtype='float')
            fault_inject_run_value = np.abs(fault_inject_run_value)

        prediction_index = np.where(fault_inject_run_value != 0)[0][1:1+knn]

        if len(prediction_index) != knn:
            continue

        correct = 0
        incorrect = 0
        for i in prediction_index:
            query_result = FAULT_INJECTION_CAMPAIGN[i*64:(i+1)*64]        

            for index, row in query_result.iterrows():
                inject_error = abs(float(row['init_value']) - float(row['corrupt_value']))
                diff = float(row['diffnorm'])

                if inject_error < fault_inject_run_value[i] and diff < THRESHOLD:
                    correct+=1
                elif inject_error < fault_inject_run_value[i] and diff >= THRESHOLD:
                    incorrect += 1

        if correct+incorrect == 0:
            prediction_accuracy.append(1)
            prediction_amount.append(0)
        else:
            prediction_accuracy.append(correct/(correct+incorrect))
            prediction_amount.append((correct + incorrect)/(knn * 64))
    print(knn)
    print(sum(prediction_accuracy)/len(prediction_accuracy))
    print(sum(prediction_amount)/len(prediction_amount))

5
0.9999928176398766
0.3296191553544492
10
0.9999854763728465
0.3253755778120188
20
0.9999799791647974
0.319899644308943
50
0.9999833664348153
0.3155008012820512
100
0.999983954317478
0.3121044019507187
200
0.9999783741352042
0.3156327110389611
300
0.9999926143025122
0.31374087712314225


In [52]:
#fault_inject_run = pd.read_csv(PROPAGATION_DATA_PATH+"/appstate_"+str(0)+".log",  sep=' ', names=['file', 'linenum', 'variable', 'value'])
#fault_inject_run_value = np.array(fault_inject_run.value, dtype='float')[0:l] - np.array(GOLDEN_RUN.value, dtype='float')
#fault_inject_run_value = np.abs(fault_inject_run_value)

#print(len(np.where(fault_inject_run_value != 0)[0]))
#print(TEST_EXPERIMENTS)
print(sum(prediction_accuracy)/len(prediction_accuracy))
print(sum(prediction_amount)/len(prediction_amount))

0.9969519556322132
0.40640998887033963


## Mitigate and Amplify Errors

1. How do you explain that case that some masked above the propagation boundary, some sdc case below the sdc cases.

In [ ]:
#FAULT_INJECTION_CAMPAIGN

# A Single Fault Injection Experiment

Understand how a single fault injection experiment can help to understand the result of the other fault injection experiment.

Randomly select 1000 fault injection experiments and test the prediction accuracy of each masked experiment.

In [ ]:
#The number of fault injection experiments that a single fault injection can predict is masked.
def single_masked_prediction(fault_inject_run, golden_run, experiments, threshold = 0.001): 
    # The experiment ends early
    if len(fault_inject_run) < len(golden_run):
        print("Bad fault injection experiment!")
        return False
    
    # Get the prediction boundary of a single fault injection experiment.
    boundary = np.abs(np.array(fault_inject_run.value[0:len(golden_run)], dtype="float") - np.array(golden_run.value, dtype="float"))    
    if np.isnan(boundary).any() or np.isinf(boundary).any():
        print("This experiment outcome is masked, but the propagation process contains nan or infinity event")
        return False
    
    # The number of tested experiment
    # print(len(experiments))
    
    positive_prediction = 0
    negative_prediction = 0
    SDC_count = 0
    crash = 0
    count = len(experiments)
  
    for index, row in experiments.iterrows():
        instruction_index = int(row["byte_num"].split("#")[1])
        inject_error = abs(float(row["corrupt_value"]) - float(row["init_value"]))
        diffnorm = float(row['diffnorm'])
        
        if inject_error <= boundary[instruction_index]:
            if diffnorm < threshold:
                positive_prediction += 1
            elif math.isnan(diffnorm) or math.isinf(diffnorm):
                crash += 1
            else:
                negative_prediction += 1     
    
    if (positive_prediction + negative_prediction) is 0:
        return {"crash":crash/count, "positive": positive_prediction/count, "negative": negative_prediction/count, "accuracy":0,  "total":positive_prediction+negative_prediction}    
    else:
        return {"crash":crash/count, "positive": positive_prediction/count, "negative": negative_prediction/count, "total":positive_prediction+negative_prediction, "accuracy":positive_prediction/(positive_prediction + negative_prediction+crash)}  

#The number of fault injection experiment that the program can predict as SDC
def single_SDC_prediction(fault_inject_run, golden_run, experiments, threshold = 0.001):
    pass

In [ ]:
# During the fault injection experiment, filter out the bad prediction case and left with the good prediction case.
GOOD_MASKED_RUN = []

In [ ]:
diffnorms = np.array(FAULT_INJECTION_CAMPAIGN.diffnorm, dtype='float')
length = len(TEST_EXPERIMENTS)
list_indexs = []
pre_res = {}
index = 0

sample_dataset = FAULT_INJECTION_CAMPAIGN.sample(n=int(TEST_EXPERIMENT_NUMBER/10), replace=False,  random_state=1)
#Understand the pruning technique.
print(len(sample_dataset))

for i, row in TEST_EXPERIMENTS.iterrows():
    file_index = row["fileindex"]
    norm = float(row["diffnorm"])
    
    
    if index > int(TEST_EXPERIMENT_NUMBER/10):
        break
    index += 1
    if index % (length/10) ==0:
        print(index/float(length), "experiment")
    
    #verify the select fault injection experiment is valuable
    if norm != 0 and not np.isinf(norm) and not math.isnan(norm) and norm < THRESHOLD :
        fault_inject_run = pd.read_csv(PROPAGATION_DATA_PATH+"/appstate_"+str(index)+".log",  sep=' ', names=['file', 'linenum', 'variable', 'value'])
        pre_res = single_masked_prediction(fault_inject_run, GOLDEN_RUN, sample_dataset, THRESHOLD)
        
        #if the predictor reject the current experiment for prediction, continue to next experiment
        if not pre_res:
            continue
        
        if pre_res['accuracy'] >= 1 :
            GOOD_MASKED_RUN.append(file_index)
        
        list_indexs.append({"diffnorm":math.log10(norm), "index":index, "crash":pre_res["crash"], "negative":pre_res["negative"], "positive":pre_res["positive"], 'accuracy':pre_res['accuracy']})


In [ ]:
GOOD_MASKED_RUN

In [ ]:
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid')
fig = plt.figure()
list_indexs = sorted(list_indexs, key = lambda i: i['diffnorm']) 
x = []
y = []
z = []
k = []
for i in range(len(list_indexs)):
    x.append(list_indexs[i]['diffnorm'])
    y.append(list_indexs[i]['positive'] + list_indexs[i]['negative'] + list_indexs[i]['crash'])
    z.append(list_indexs[i]['negative'])
    k.append(list_indexs[i]['accuracy'])

In [ ]:
sns.set(style="ticks")
sns.axes_style('white')
df = pd.DataFrame(data = {"x":x, "y":y, "k":k,"z":z})

plt.scatter(x, y, c=k, cmap='viridis')
plt.xlabel('norm(log10)')
plt.ylabel('prediction rate')
#plt.clim(0,1)
plt.colorbar()
plt.show()

In [ ]:
plt.subplot(2, 2, 1)
plt.hist(df.x)
plt.xlabel('norm')
plt.ylabel('number of experiments')

plt.subplot(2, 2, 2)
plt.hist(df.y)
plt.xlabel('positive prediction')

plt.subplot(2, 2, 3)
plt.hist(df.k)
plt.xlabel('prediction accuracy')

plt.subplot(2, 2, 4)
plt.hist(df.z)
plt.xlabel('negative prediction')

# We use 1000 experiments as a filter to filter out outlier propagation data

1. here is a very interesting, we use the subset of the sample to filter the outlier propagation data. Why this type of data is outlier

In [76]:
tracking_length = len(GOLDEN_RUN.value)
def construct_boundary(cases):
    boundary = []
    for i in range(tracking_length):
        boundary.append(0)

    for i in range(len(cases)):
        index = cases[i]
        #masked_run_path = "../static/data/cg/cg_in8/appstate_"+ str(index)+".log" 
        masked_run_path = PROPAGATION_DATA_PATH +"/appstate_"+ str(index) + ".log"
        #Check whether the tracking file is on the path
        #TODO: check whether can regenerate the data
        #print(masked_run_path)
        if not os.path.isfile(masked_run_path):
            print("file does not exist")
            continue

        masked_run = pd.read_csv(masked_run_path,  sep=" ", names=['file', 'linenum', 'variable', 'value'])
        masked_run_value = np.array(masked_run.value, dtype='float')
        
        #This is a werid information in the data.
        #TODO: may check why such outcome is shown
        if len(masked_run_value) < tracking_length:
            print("odd!")
            continue

        #array contain NAN, ignore the run
        if np.isnan(np.min(masked_run_value)):
            continue

        comparision_result = np.abs(masked_run_value[0:tracking_length] - GOLDEN_RUN.value)
        for j in range(tracking_length):
            if comparision_result[j] > boundary[j]:
                boundary[j] = comparision_result[j]
                
    return boundary

def predict(boundary):
    positive_prediction = 0
    crash = 0
    negative_prediction = 0
    result = {}
    bits = []

    for index, row in FAULT_INJECTION_CAMPAIGN.iterrows():
        instruction_index = int(row['byte_num'].split('#')[1])#int(row["DI"]) 
        
        inject_error = abs(float(row['init_value']) - float(row['corrupt_value']))#abs(row["out_xor"])
        diffnorm = float(row['diffnorm'])
        
        if inject_error <= boundary[instruction_index]:
            if diffnorm < THRESHOLD:
                positive_prediction += 1
            elif math.isnan(diffnorm) or math.isinf(diffnorm):
                crash += 1
            else:
                negative_prediction += 1     
    
    result["positive"] = positive_prediction
    result["negative"] = negative_prediction
    result["crash"] = crash
    result["accuracy"] = positive_prediction/(positive_prediction + crash + negative_prediction)
    
    return (result)

In [ ]:
FAULT_INJECTION_CAMPAIGN

In [ ]:
#import dill
#dill.load_session('notebook_env.db')

In [ ]:
#print(GOOD_MASKED_RUN)
#boundary = construct_boundary(GOOD_MASKED_RUN)

In [77]:
#res, bits = predict(boundary)
res = predict(boundary)

#plt.hist(bits)
print(res)
#plt.hist(df.x)
#plt.xlabel('norm')
#plt.ylabel('number of experiments')

NameError: name 'boundary' is not defined

In [ ]:
#plot the boundary
#plt.plot(boundary)

# Extracting A single fault injection site to understand the error propagation.